In [37]:
from random import choices
from datetime import datetime
from datetime import date
from tinydb import TinyDB, Query
import sys
import json
import logging

logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.INFO)

date_format = "%d.%m.%Y"
dbname = "colleagues.json"
logger.info("Using file %s", dbname)
db = TinyDB(dbname)


INFO:root:Using file colleagues.json


In [38]:
class InputHelper:    
    def getNextCoffeeDate():
        coffee_date = None
        while not coffee_date:
            coffee_date_text = input("Termin setzen am dd.mm.YYYY!")
            if(len(coffee_date_text) > 0):
                try:
                    temp_coffee_date = datetime.strptime(coffee_date_text, date_format)

                    if((temp_coffee_date - datetime.today()).days >= 0):
                        coffee_date = temp_coffee_date
                    else:
                        logger.info("%s is in the past", coffee_date_text)

                except ValueError:
                    logger.info("Value %s is not a valid date", coffee_date_text)
            else:
                break

        return coffee_date

In [39]:
class Employee:
    def __init__(self, name, company, willing, last_coffee):
        self.name = name
        self.company = company
        self.willing = willing
        self.last_coffee = last_coffee

    def weight(self):
        temp_weight = self.__days_since_last_coffee() + 3
        
        if(self.company != "Verimi"):
            temp_weight = temp_weight * 0.7    
        if(self.willing):
            temp_weight = temp_weight * 1.5

        if(temp_weight <= 0):
            temp_weight = 0

        return temp_weight

    def __days_since_last_coffee(self):
        return (datetime.today() - datetime.strptime(self.last_coffee, date_format)).days


In [40]:
names = []
weights = []
winners = []

for entry in db.all():
    e = Employee(**entry)

    names.append(e.name)
    weights.append(e.weight())
    logger.info('%s: %s', e.name, e.weight())

if(len(names) > 4):
    while(len(winners) < 4):
        next = choices(names, weights = weights, k = 1)
        if(next[0] not in winners):
            winners.append(next[0])
else:
    logger.warn("Not enough coffee candidates")
    sys.exit()

print("")
print("########################################################################")
print("Have coffee with")
print(winners)
print("#############################d###########################################")
print("")

coffee_date = InputHelper.getNextCoffeeDate()
if coffee_date:
    for e_name in winners:
        date_for_db = coffee_date.strftime(date_format)
        db.update({'last_coffee': date_for_db}, Query().name == e_name)
        logger.info("Coffee Date %s stored to database for %s", date_for_db, e_name)
else:
    logger.warn("Coffee date was not updated")

INFO:root:Patrick: 7.5
INFO:root:Liang: 4
INFO:root:Sukhmeet: 6.0
INFO:root:Jan: 0
INFO:root:Hadi: 4.5
INFO:root:Manju: 0
INFO:root:Oleh: 0
INFO:root:Svitlana: 6.0
INFO:root:Gurpreet: 15.0
INFO:root:Chidubem: 0
INFO:root:Bharat: 15.0
INFO:root:Tapan: 3
INFO:root:Tarun: 3
INFO:root:Sandor: 0
INFO:root:Andras B.: 0
INFO:root:Andrea: 4.8999999999999995
INFO:root:Szilvia: 2.0999999999999996
INFO:root:Andras S.: 0
INFO:root:Peter: 3.1499999999999995
INFO:root:Mirjam: 7.35
INFO:root:Oleks: 0
INFO:root:Lajos: 7.35
INFO:root:Miklos: 0
INFO:root:Bernat: 0
INFO:root:Biju: 4.5
INFO:root:Mohamed: 0
INFO:root:Marius: 0
INFO:root:Thierry: 4.5

########################################################################
Have coffee with
['Gurpreet', 'Lajos', 'Mirjam', 'Svitlana']
#############################d###########################################

